<a href="https://colab.research.google.com/github/yukinaga/neural_network_on_torus/blob/master/neural_network_on_torus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural network on a torus

## Importing packages

In [0]:
import numpy as np
from PIL import Image, ImageDraw
import IPython.display as disp

## A class of neural network on a torus

In [0]:
class TorusNetwork():
    def __init__(self, n_h, n_w, n_connect):
        n_neuron = n_h * n_w        
        self.params = (n_h, n_w, n_neuron, n_connect)

        self.connect_ids = None
        self.w = None
        self.b = None
        self.y = None
        self.proj = None
        self.proj_ids = None
        self.proj_to_ids = None
        self.inhib = None

    def connect(self, proj_ratio, sigma_inter):
        n_h, n_w, n_neuron, n_connect = self.params
        
        n_proj= int(proj_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        self.proj_ids = rand_ids[:n_proj]
        self.proj_to_ids = np.random.permutation(self.proj_ids)
        self.proj = np.zeros(n_neuron, dtype=np.bool)
        self.proj[self.proj_ids] = True
                
        inter_dist_x = np.random.randn(n_neuron, n_connect) * sigma_inter
        inter_dist_x = np.where(inter_dist_x<0, inter_dist_x-0.5, inter_dist_x+0.5).astype(np.int32) 
        x_connect = np.zeros((n_neuron, n_connect), dtype=np.int32)
        x_connect += np.arange(n_neuron).reshape(-1, 1)
        x_connect %= n_w
        x_connect += inter_dist_x
        x_connect = np.where(x_connect<0, x_connect+n_w, x_connect)
        x_connect = np.where(x_connect>=n_w, x_connect-n_w, x_connect)
            
        inter_dist_y = np.random.randn(n_neuron, n_connect) * sigma_inter
        inter_dist_y = np.where(inter_dist_y<0, inter_dist_y-0.5, inter_dist_y+0.5).astype(np.int32)        
        y_connect = np.zeros((n_neuron, n_connect), dtype=np.int32)
        y_connect += np.arange(n_neuron).reshape(-1, 1)
        y_connect //= n_w
        y_connect += inter_dist_y
        y_connect = np.where(y_connect<0, y_connect+n_h, y_connect)
        y_connect = np.where(y_connect>=n_h, y_connect-n_h, y_connect)        

        self.connect_ids = x_connect + n_w * y_connect
        
    def initialize_network(self, inhib_ratio, w_mu, w_sigma):
        n_h, n_w, n_neuron, n_connect = self.params

        n_inhib = int(inhib_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        inhib_ids = rand_ids[:n_inhib]
        self.inhib = np.zeros(n_neuron, dtype=np.bool)
        self.inhib[inhib_ids] = True
        
        self.w = np.random.randn(n_neuron, n_connect) * w_sigma + w_mu
        self.w = np.where(np.isin(self.connect_ids, inhib_ids), -self.w, self.w)
        self.w /= np.sum(self.w, axis=1, keepdims=True)
       
        self.b = np.full(n_neuron, 0.5)
        
        self.y = np.random.randint(0, 2, n_neuron, dtype=np.bool)

    def forward(self, delta_b, ramda_w):
        self.y[self.proj_to_ids] = self.y[self.proj_ids] 
        x = self.y[self.connect_ids]
        u = np.sum(self.w * x, axis=1) - self.b
        self.y = np.where(u<0, False, True)
        
        self.b = np.where(self.y, self.b+delta_b, self.b-delta_b)
        self.b *= np.sum(self.y) / np.sum(self.b) 
        
        self.w += ramda_w * self.y.reshape(-1, 1) * x * np.where(self.w<0, -1, 1)
        self.w /= np.sum(self.w, axis=1, keepdims=True)

## Settings

In [0]:
n_h = 128
n_w = n_h * 4
n_connect = 64
tnet = TorusNetwork(n_h, n_w, n_connect)

proj_ratio = 0.25
sigma_inter = 4
tnet.connect(proj_ratio, sigma_inter)

inhib_ratio = 0.2
w_mu = 0.2
w_sigma = 0.08

delta_b = 0.01
ramda_w = 0.001

frames = 360

## Temporal change of neurons

In [0]:
tnet.initialize_network(inhib_ratio, w_mu, w_sigma)

c_proj_exc = np.array([0, 0, 255]).reshape(1, -1)
c_proj_inh = np.array([255, 0, 0]).reshape(1, -1)
c_inter_exc = np.array([30, 144, 255]).reshape(1, -1)
c_inter_inh = np.array([255, 105, 180]).reshape(1, -1)

images = []
for i in range(frames):
    tnet.forward(delta_b, ramda_w)
    
    y = tnet.y.reshape(-1, 1)
    proj = tnet.proj.reshape(-1, 1)
    inhib = tnet.inhib.reshape(-1, 1)

    image = np.zeros((n_h*n_w, 3))
    image = np.where(y & proj & ~inhib, c_proj_exc, image)
    image = np.where(y & proj & inhib, c_proj_inh, image)
    image = np.where(y & ~proj & ~inhib, c_inter_exc, image)
    image = np.where(y & ~proj & inhib, c_inter_inh, image)
    image = image.reshape(n_h, n_w, -1).astype(np.uint8)

    image = Image.fromarray(image)
    images.append(image)

images[0].save('tnet_movie.gif',
                   save_all=True, append_images=images[1:], optimize=False, duration=100, loop=0)
with open('tnet_movie.gif','rb') as f:
    disp.display(disp.Image(f.read()))

## Two-dementional outputs of two torus networks (not connected)

In [0]:
tnet1 = TorusNetwork(n_h, n_w, n_connect)
tnet1.connect(proj_ratio, sigma_inter)
tnet1.initialize_network(inhib_ratio, w_mu, w_sigma)

tnet2 = TorusNetwork(n_h, n_w, n_connect)
tnet2.connect(proj_ratio, sigma_inter)
tnet2.initialize_network(inhib_ratio, w_mu, w_sigma)

c_out1 = np.array([0, 191, 255])
c_out2 = np.array([255, 165, 0])

images = []
pre_image = np.zeros((frames, n_h*2, 3), dtype=np.uint8)
for i in range(frames):
    tnet1.forward(delta_b, ramda_w)
    y1 = tnet1.y.reshape(n_h, n_w)
    
    tnet2.forward(delta_b, ramda_w)
    y2 = tnet2.y.reshape(n_h, n_w)
    
    image = np.zeros((n_h, n_h*2, 3), dtype=np.uint8)
    image[:, :n_h, :] = y1[:, 0:n_h].reshape(n_h, n_h, 1) * c_out1.reshape(1, 1, -1)
    image[:, n_h:, :] = y2[:, 0:n_h].reshape(n_h, n_h, 1) * c_out2.reshape(1, 1, -1)
    
    image = Image.fromarray(image)
    images.append(image)

images[0].save('communication_not_connected.gif',
                   save_all=True, append_images=images[1:], optimize=False, duration=100, loop=0)
print("Not connected")
with open('communication_not_connected.gif','rb') as f:
    disp.display(disp.Image(f.read()))

## Two-dementional outputs of two connected torus networks

In [0]:
tnet1 = TorusNetwork(n_h, n_w, n_connect)
tnet1.connect(proj_ratio, sigma_inter)
tnet1.initialize_network(inhib_ratio, w_mu, w_sigma)

tnet2 = TorusNetwork(n_h, n_w, n_connect)
tnet2.connect(proj_ratio, sigma_inter)
tnet2.initialize_network(inhib_ratio, w_mu, w_sigma)

c_out1 = np.array([0, 191, 255])
c_out2 = np.array([255, 165, 0])

images = []
pre_image = np.zeros((frames, n_h*2, 3), dtype=np.uint8)
for i in range(frames):
    tnet1.forward(delta_b, ramda_w)
    y1 = tnet1.y.reshape(n_h, n_w)
    
    tnet2.forward(delta_b, ramda_w)
    y2 = tnet2.y.reshape(n_h, n_w)
    
    y2[:, n_h*2:n_h*3] = y1[:, :n_h]
    y1[:, n_h*2:n_h*3] = y2[:, :n_h]
    
    image = np.zeros((n_h, n_h*2, 3), dtype=np.uint8)
    image[:, :n_h, :] = y1[:, 0:n_h].reshape(n_h, n_h, 1) * c_out1.reshape(1, 1, -1)
    image[:, n_h:, :] = y2[:, 0:n_h].reshape(n_h, n_h, 1) * c_out2.reshape(1, 1, -1)
    
    image = Image.fromarray(image)
    images.append(image)

images[0].save('communication_connected.gif',
                   save_all=True, append_images=images[1:], optimize=False, duration=100, loop=0)
print("Connected")
with open('communication_connected.gif','rb') as f:
    disp.display(disp.Image(f.read()))